# 1. 패키지 설치

In [1]:
%pip install python-dotenv langchain langchain-upstage langchain-community langchain-text-splitters docx2txt langchain-chroma

Note: you may need to restart the kernel to use updated packages.


# 2. Knowledge Base 구성을 위한 데이터 생성

- [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/)를 활용한 데이터 chunking
    - split 된 데이터 chunk를 Large Language Model(LLM)에게 전달하면 토큰 절약 가능
    - 비용 감소와 답변 생성시간 감소의 효과
    - LangChain에서 다양한 [TextSplitter](https://python.langchain.com/v0.2/docs/how_to/#text-splitters)들을 제공
- `chunk_size` 는 split 된 chunk의 최대 크기
- `chunk_overlap`은 앞 뒤로 나뉘어진 chunk들이 얼마나 겹쳐도 되는지 지정

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_docs/tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [5]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

# 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [7]:
retrieved_docs

[Document(metadata={'source': './tax_docs/tax_with_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n종합소득 과세표준\n\n세율\n\n1,400만원 초과\n\n과세표준의 6퍼센트\n\n1,400만원 초과 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15퍼센트)\n\n5,000만원 초과 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24퍼센트)\n\n8,800만원 초과 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)\n\n1억5천만원 초과 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)\n\n3억원 초과 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40퍼센트)\n\n5억원 초과 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45퍼센트)\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 31.>\n\n\n\n제56조(배당세액공제) 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [8]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
ai_message = qa_chain({"query": query})

C:\Users\김경덕\AppData\Local\Temp\ipykernel_28520\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [12]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [15]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만 원인 직장인의 소득세는 1,217,500원입니다.'}